# 3.3 线性回归的简洁实现

In [1]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.9.0


## 3.3.1 生成数据集

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = torch.matmul(features, torch.Tensor(true_w))+true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 3.3.2 读取数据

In [3]:
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

In [4]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[-0.3562,  0.2133],
        [ 0.4001, -0.0795],
        [ 1.0890, -0.1694],
        [-0.1943, -1.6204],
        [-1.1913, -0.7832],
        [ 2.7207, -0.2510],
        [ 1.1821, -0.2185],
        [ 0.7352,  0.5962],
        [-0.6132,  0.1072],
        [-1.0462, -1.3338]]) 
 tensor([ 2.7522,  5.2682,  6.9627,  9.3293,  4.4788, 10.4889,  7.3042,  3.6320,
         2.6205,  6.6509])


## 3.3.3 定义模型

In [9]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super().__init__()
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [6]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [10]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1025, -0.0028]], requires_grad=True)
Parameter containing:
tensor([0.6181], requires_grad=True)


## 3.3.4 初始化模型参数

In [16]:
net.linear.weight, net.linear.bias

(Parameter containing:
 tensor([[ 0.1025, -0.0028]], requires_grad=True),
 Parameter containing:
 tensor([0.6181], requires_grad=True))

In [27]:
from torch.nn import init

init.normal_(net.linear.weight, mean=0.0, std=0.01)
init.constant_(net.linear.bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [28]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.0057, 0.0115]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5 定义损失函数

In [29]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [30]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [31]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [32]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [36]:
num_epochs = 5
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print(output.size())
    print('epoch %d, loss: %f' % (epoch, l.item()))

torch.Size([10, 1])
epoch 1, loss: 0.000097
torch.Size([10, 1])
epoch 2, loss: 0.000064
torch.Size([10, 1])
epoch 3, loss: 0.000137
torch.Size([10, 1])
epoch 4, loss: 0.000173
torch.Size([10, 1])
epoch 5, loss: 0.000097


In [34]:
dense = net.linear
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 1.9996, -3.4004]])
4.2 tensor([4.1996])
